In [8]:
using Plots

In [2]:
function error_metric(xaprox::Float64, xtarget::Float64, is_absolute::Bool)
    
    """
        This function calculates the error between the aproximation and the real value
        

        Parameters
        ----------
        xaprox: Float64, The aproximation of the of the function.
        xtarget: Float64, The real value we want.
        is_absolute: Bool, If true then the error is absolute, if false then the error is relative.

        Returns
        -------
        Float64, The error between the aproximation and the real value of the root of the function.
    """
    
    if is_absolute
        return abs(xtarget - xaprox) 
    else
        return abs(xtarget - xaprox)/xtarget
    end
    
end

error_metric (generic function with 1 method)

In [3]:
function runge_kutta(f, x0, y0, h, n, tol)
    x = x0
    y = y0
    for i in 1:n
        k1 = h * f(x, y)
        k2 = h * f(x + h / 2, y + k1 / 2)
        k3 = h * f(x + h / 2, y + k2 / 2)
        k4 = h * f(x + h, y + k3)
        y_new = y + (k1 + 2 * k2 + 2 * k3 + k4) / 6
        x_new = x + h

        # Error control
        while abs(y_new - y) > tol
            h = h / 2
            k1 = h * f(x, y)
            k2 = h * f(x + h / 2, y + k1 / 2)
            k3 = h * f(x + h / 2, y + k2 / 2)
            k4 = h * f(x + h, y + k3)
            y_new = y + (k1 + 2 * k2 + 2 * k3 + k4) / 6
            x_new = x + h
        end

        y = y_new
        x = x_new
    end
    return y
end

runge_kutta (generic function with 1 method)

![Alt text](image-2.png)

In [4]:
function runge_kutta_iteration(f, x, y, h, verbose = false)
    k1 = h * f(x, y)
    k2 = h * f(x + h / 4, y + 1/4*k1)
    k3 = h * f(x + 3 * h / 8, y + 3 * k1 / 32 + 9 * k2 / 32)
    k4 = h * f(x + 12 * h / 13, y + 1932 * k1 / 2197 - 7200 * k2 / 2197 + 7296 * k3 / 2197)
    k5 = h * f(x + h, y + 439 * k1 / 216 - 8 * k2 + 3680 * k3 / 513 - 845 * k4 / 4104)
    k6 = h * f(x + h / 2, y - 8 * k1 / 27 + 2 * k2 - 3544 * k3 / 2565 + 1859 * k4 / 4104 - 11 * k5 / 40)

    if verbose
        println("k1: ", k1)
        println("k2: ", k2)
        println("k3: ", k3)
        println("k4: ", k4)
        println("k5: ", k5)
        println("k6: ", k6)
    end

    y_new = y + 25 * k1 / 216 + 1408 * k3 / 2565 + 2197 * k4 / 4104 - k5 / 5
    y_high_ord_new = y + 16 * k1 / 135 + 6656 * k3 / 12825 + 28561 * k4 / 56430 - 9 * k5 / 50 + 2 * k6 / 55

    if verbose
        println("y_new: ", y_new)
        println("y_high_ord_new: ", y_high_ord_new)
        println("R: ", error_metric(y_new, y_high_ord_new, true))
    end

    return y_new, y_high_ord_new
end

runge_kutta_iteration (generic function with 2 methods)

In [5]:
function runge_kutta_fehlberg(f, x0, y0, h_max, h_min, n, tol, is_absolute)
    x = x0
    y = y0 

    points = []
    h = h_max
    count = 0

    for i in 1:n
        
        if count == 0
            verbose = true
            count = 1
        else
            verbose = false
        end

        y_new, y_high_ord_new = runge_kutta_iteration(f, x, y, h, verbose)
        x_new = x + h
        R = error_metric(y_new, y_high_ord_new, is_absolute)        
        while  R > tol
            q = 0.84* ((tol*h)/(R))^(1/4) 
            h = q*h
            
            if h < h_min
                error("h_min is too high")
            end

            y_new, y_high_ord_new = runge_kutta_iteration(f, x_new, y_new, h)
            x_new = x_new + h
            R = error_metric(y_new, y_high_ord_new, is_absolute)
        end
        y = y_new
        x = x_new

        push!(points, [x, y])
    end

     return points
end

runge_kutta_fehlberg (generic function with 1 method)

In [6]:
y′ = (y,t) -> y - t^2 + 1
tol = 10^-5
x0 = 0.5  #  -> y 
y0 = 0  #  -> t
h_max = 0.25
h_min = 0.01


points = runge_kutta_fehlberg(y′, x0, y0, h_max, h_min, 100, tol, true)

k1: 0.375
k2: 0.388427734375
k3: 0.3932245480682468
k4: 0.3998098738643211
k5: 0.39884776086886387
k6: 0.3968550498483333
y_new: 0.3935159143876452
y_high_ord_new: 0.39351748233252104
R: 1.5679448758243453e-6


100-element Vector{Any}:
 [0.75, 0.3935159143876452]
 [1.0, 0.7730034468636143]
 [1.3840247190148085, 1.2161634701548147]
 [1.518049438029617, 1.327217516795898]
 [1.6520741570444255, 1.4201759136741705]
 [1.786098876059234, 1.4986596925718634]
 [1.9201235950740425, 1.5659771756987264]
 [2.054148314088851, 1.6248860006697687]
 [2.1881730331036593, 1.6775507593852446]
 [2.322197752118468, 1.7256013817078608]
 ⋮
 [13.312224711332783, 3.765475394447606]
 [13.446249430347592, 3.7833144106799685]
 [13.5802741493624, 3.8010685013709793]
 [13.71429886837721, 3.818738879681225]
 [13.848323587392018, 3.8363267299351302]
 [13.982348306406827, 3.8538332085779965]
 [14.116373025421636, 3.8712594450923348]
 [14.250397744436444, 3.8886065428756065]
 [14.384422463451253, 3.9058755800813447]